In [1]:
#!/usr/bin/env python3

import pickle
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.axes as axs
from prettytable import PrettyTable
from operator import add

from dataTy import dataTy
from dataTy import Output
from dataTy import ResultHelper

def Round(F):
    return round(F,3)
def open_pickle(default_name):
    pfile = default_name
    # Use this in terminal py
    #if len(sys.argv) > 1:
        # first arg is filename
    #    pfile = sys.argv[1]
    print("Opening " + pfile)
    with open(pfile,'rb') as f:
        result = pickle.load(f)
    return result

class Summarizer:
    def __init__(self, result):
        PT = PrettyTable()
        fields = [ '\\' ]
        for config in ResultHelper.getConfigs(result):
            fields.append(config)
        PT.field_names = fields
        for proj in ResultHelper.getProjs(result):
            row = [proj]
            for config in ResultHelper.getConfigs(result):
                avg_time = Round(ResultHelper.getErrorOrAvgTime(result[config][proj]))
                row.append(avg_time)
            PT.add_row(row)
        print(PT)
        
class Printer:
    def __init__(self, result, the_config = None, the_proj = None):
        if the_config == None :
             configs = result.keys()
        else:
            configs = [the_config]
        for config in configs:
            output_of_proj = result[config]
            if the_proj == None:
                projs = output_of_proj.keys()
            else:
                projs = [the_proj]
                    
            for proj in projs:
                # Print with PrettyTable
                output = output_of_proj[proj]
                PT = PrettyTable()
                T = ResultHelper.getErrorOrAvgTime(output)
                PT.title = "" + config + " " + proj + " " + str(Round(T))
                PT.field_names = ["Attr", "Time", "Count"]
                PT.align["Attr"] = "l"
                PT.add_row(["Total", Round(output.prof_time), ""])
                for name in output.prof_data:
                    d = output.prof_data[name]
                    PT.add_row([name, Round(d.Value), d.Count])
                print(PT)
class config:
    width = 1
    margin = 1
    left_margin = 1
    def rename_config(config):
        renametable = {"omp-offload" : "clang", "omp-offload-1d" : "1D", "omp-offload-bulk": "bulk", "omp-offload-at": "bulk+at"}
        ret = renametable.get(config)
        if ret == None:
            return config
        else:
            return ret
    save_img = True

class PlotPrinter:
    def submit(name):
        # Make layout clear
        plt.tight_layout()
        
        # Add hlines
        [ymin, ymax] = plt.gca().get_ylim()
        if ymax > 100:
            step = 10
        else:
            step = (ymax - ymin)/ 8
        
        ylist = np.arange(0, ymax, step)
        [xmin, xmax] = plt.gca().get_xlim()
        plt.hlines(ylist, xmin, xmax, zorder=0, linewidth=1)
        
        if config.save_img:
            plt.savefig("plot/" + name + ".png", format='png',dpi=300, edgecolor='k')
        else:
            plt.show()
        plt.clf()

class ColumnChartPrinter:
    def plot(result, norm=False):
        # Get projs
        projs = ResultHelper.getProjs(result)
        configs = ResultHelper.getConfigs(result)
        proj_count = len(projs)
        config_count = len(configs)
        if ResultHelper.invalid(result) :
            return
        n = 0
        xpos_base = np.arange(0, proj_count * (config_count + 1), config_count + 1)
        last_col = xpos_base[-1]
        
        factors =  ResultHelper.getNormFactors(result, ["Times"], norm)
        for c in configs:
            # prepare data
            height = []
            xpos = [x + config.width * n for x in xpos_base]
            for p in projs:
                time = result[c][p].prof_data["Times"].Value * factors[p]
                height.append(time)
            plt.bar(xpos, height, width=config.width, label=c)
            last_col = xpos[-1]
            n += 1
        if norm == True:
            plt.ylabel('Normalized Execution Time (%)', fontweight='bold')
        else:
            plt.ylabel('Execution Time(sec)', fontweight='bold')
        
        plt.xlabel('Benchmarks', fontweight='bold')
        plt.title('Overview', fontweight='bold')
        # Insert in the center
        xticks = [x + config_count / 2 * config.width for x in xpos_base]
        plt.xticks(xticks, projs)
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=2)
        plt.xlim(xpos_base[0] - config.left_margin, last_col + config.margin)
        PlotPrinter.submit("all")

class StackChartPrinter:
    def plotStackChar(result, configs, proj, ind, metrics):
        N = len(configs)
        btn = [0] * N
        plist = []
        for m in metrics:
            time = []
            for c in configs:
                time.append(float(result[c][proj].prof_data[m].Value))
            p = plt.bar(ind, time, config.width, bottom=btn)
            plist.append(p)
            # Increase btn
            btn = list(map(add, btn, time))
        return plist

    def plot(result, proj, norm=False):
        configs = ResultHelper.getConfigs(result)
        metrics = ["Other","OMPRuntime", "Kernel", "H2DTransfer", "D2HTransfer", "UpdatePtr"]
        #metrics = ["OMPRuntime", "Kernel", "H2DTransfer", "D2HTransfer", "UpdatePtr"]
        
        config_count = len(configs)
        ind = np.arange(0, config_count * (config.width + 1),  config.width + 1)
        #ind = np.arange(N)    # the x locations for the groups

        plist = StackChartPrinter.plotStackChar(result, configs, proj, ind, metrics)

        plt.ylabel('Execution Time(sec)', fontweight='bold')
        plt.xlabel('Configs', fontweight='bold')
        plt.title('[Breakdown] ' + proj, fontweight='bold')
        plt.xticks(ind, [config.rename_config(x) for x in configs]) #plt.yticks(np.arange(0, 81, 10))
        plt.legend(plist, metrics, loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=3)
        plt.xlim(ind[0]- config.left_margin, ind[-1] + config.margin)
        PlotPrinter.submit(proj)

class OffloadVS1DNormPrinter:
    # Like stackChart
    def plot(result):
        projs = ResultHelper.getProjs(result)
        configs = ["omp-offload", "omp-offload-1d"]
        metrics = ["OMPRuntime", "Kernel", "H2DTransfer", "D2HTransfer", "UpdatePtr"]
        for c in configs:
            if result.get(c) == None:
                print(c + " key not found")
                return
        # Gen xpos and xticks
        xpos = []
        xticks = []
        ind = np.arange(0, 2 * (config.width + 0.5), config.width + 0.5).tolist() # the x locations for the groups
        for proj in projs:
            xpos = xpos + ind
            xticks += [proj, proj+"1d"]
            ind = [x + 2 * config.width + 2.5 * config.width for x in ind]

        btn = [0] * len(projs) * 2
        plist = []
        # calculate the factor
        factors =  ResultHelper.getNormFactors(result, metrics, True)
                
        for m in metrics:
            time = []
            for proj in projs:
                f = factors[proj]
                for c in configs:
                    time.append(f*float(result[c][proj].prof_data[m].Value))
                    
            plist.append(plt.bar(xpos, time, config.width, bottom=btn))
            btn = list(map(add, btn, time))
        plt.ylabel('Execution Time(%)', fontweight='bold')
        plt.ylabel('Normalized Execution Time (%)', fontweight='bold')
        #plt.xlabel('Configs', fontweight='bold')
        plt.title("Direct Offload vs 1D Refactor Comparison", fontweight='bold')
        # shift because of rotation
        plt.xticks([x - config.width/2 for x in xpos], xticks, rotation=40)
        plt.legend(plist, metrics, loc='upper center', bbox_to_anchor=(0.5, -0.4), ncol=3)
        plt.xlim(xpos[0] - config.left_margin, xpos[-1] + config.margin)
        PlotPrinter.submit("Offloadvs1DCompare")

class ATOptPrinter:
    def plot(results):
        for r in results:
            first = results[r]
            break
        opts = results.keys()
        
        projs = ResultHelper.getProjs(first)
        the_config = "omp-offload-at"
        #metrics = [ "Kernel"]
        metrics = ["OMPRuntime", "Kernel", "H2DTransfer", "D2HTransfer", "UpdatePtr"]
        
        # Gen xpos and xticks
        xpos = []
        xticks = []
        ind = np.arange(0, 2 * (config.width + 0.5), config.width + 0.5).tolist() # the x locations for the groups
        for proj in projs:
            xpos = xpos + ind
            xticks += [proj, proj+"-opt"]
            ind = [x + 2 * config.width + 2.5 * config.width for x in ind]

        btn = [0] * len(projs) * 2
        plist = []
        # calculate the factor
        factors = {}
        for proj in projs:
            sum = 0
            for m in metrics:
                sum += float(first[the_config][proj].prof_data[m].Value)
            factors[proj] = 100/sum
                
        for m in metrics:
            time = []
            for proj in projs:
                f = factors[proj]
                for o in opts:
                    time.append(f*float(results[o][the_config][proj].prof_data[m].Value))
                    
            plist.append(plt.bar(xpos, time, config.width, bottom=btn))
            btn = list(map(add, btn, time))
        plt.ylabel('Execution Time(%)', fontweight='bold')
        plt.title("AT Opt Comparison", fontweight='bold')
        # shift because of rotation
        plt.xticks([x - config.width/2 for x in xpos], xticks, rotation=40)
        plt.legend(plist, metrics, loc='upper center', bbox_to_anchor=(0.5, -0.4), ncol=3)
        plt.xlim(xpos[0] - config.left_margin, xpos[-1] + config.margin)
        #plt.ylim(top=110)
        PlotPrinter.submit("ATOptEffect")
        
# Compare different at optimization
def at_opt():
    files = {}
    files["O0"]   = "./results/at-O0.p"
    files["Osm"]  = "./results/at-Osm.p"
    
    results = {}
    for f in files:
        results[f] = pickle.load(open(files[f], "rb"))
        print("Open "+ files[f])
        ResultHelper.preprocessing(results[f])
    ATOptPrinter.plot(results)
    
def main():
    # Read from pickle
    result = open_pickle("./results/result.p")
    #result = open_pickle("./results/Result-all-3.p")
    ResultHelper.preprocessing(result)
    Summarizer(result)
    #return
    Printer(result)
    for config in result:
        for proj in result[config]:
            print()
 
    return 
    ColumnChartPrinter.plot(result,True)
    OffloadVS1DNormPrinter.plot(result) 
    for p in ResultHelper.getProjs(result):
        StackChartPrinter.plot(result, p)
        pass
        

plt.rcParams['figure.dpi'] = 80
config.save_img = False
plt.style.use('seaborn-deep') # choose style https://matplotlib.org/3.1.0/gallery/style_sheets/style_sheets_reference.html

if __name__ == "__main__":
    main()
    #at_opt()
    

Opening ./results/result.p
+----------+----------------+
|    \     | omp-offload-1d |
+----------+----------------+
| backprop |      0.33      |
+----------+----------------+
+----------------------------------------------+
|         omp-offload-1d backprop 0.33         |
+-----------------------------+--------+-------+
| Attr                        |  Time  | Count |
+-----------------------------+--------+-------+
| Total                       |  0.31  |       |
| Runtime                     | 0.126  |   12  |
| Kernel                      | 0.109  |   4   |
| H2DTransfer                 | 0.009  |   14  |
| UpdatePtr                   |  0.0   |   0   |
| D2HTransfer                 | 0.007  |   10  |
| RTTarget                    | 0.109  |   4   |
| RTDataBegin                 | 0.009  |   4   |
| RTDataUpdate                |  0.0   |   0   |
| RTDataEnd                   | 0.008  |   8   |
| Parallelism                 | 4352.0 |   4   |
| ATTableSize                 |  0.0   

In [2]:
import matplotlib.pyplot as plt
from IPython.display import HTML, display
import tabulate

def TableGen(result, proj):
    colLabels = ["[{0}]".format(proj)]
    rowLabelsDict = {}
    index = 0
    for config in result:
        colLabels.append(config)
        output = result[config][proj].prof_data
        for attr in output:
            attr_count = attr + "_count"
            # init attr
            if rowLabelsDict.get(attr) == None:
                rowLabelsDict[attr] = [attr]
                rowLabelsDict[attr_count] = ["-"]
            CountDiff = index - len(rowLabelsDict[attr])
            for i in range(CountDiff):
                print("Append lost")
                rowLabelsDict[attr].append("NaN")
                rowLabelsDict[attr_count].append("NaN")
            rowLabelsDict[attr].append(output[attr].Value)
            rowLabelsDict[attr_count].append(output[attr].Count)
        index = index + 1
    # Merge data to list
    clust_data = []
 
    for entry in rowLabelsDict:
        clust_data.append(rowLabelsDict[entry])
    
    #StackChartPrinter.plot(result, proj)
    display(HTML("<style>table, th, td {font-size: 15px}</style>" + tabulate.tabulate(clust_data, colLabels, tablefmt='html')))

def main():
    # Data
    result = open_pickle("./results/result.p")
    #result = open_pickle("./results/Result-all-3.p")
    ResultHelper.preprocessing(result)
    TableGen(result, "backprop")

main()


Opening ./results/result.p


[backprop],omp-offload-1d
Runtime,0.12554
-,12
Kernel,0.108891
-,4
H2DTransfer,0.008523
-,14
UpdatePtr,0
-,0
D2HTransfer,0.007318
-,10
